# MLOps with CLI V2

- Creating Scripts for MLOps Pipeline


In [15]:
#pip install mlflow azureml-mlflow
#https://learn.microsoft.com/en-us/azure/machine-learning/how-to-log-view-metrics?tabs=interactive

In [29]:
# subscription_id = '<SUBSCRIPTION_ID>'
# resource_group= '<resource_group>'
# workspace = '<workspace>'

subscription_id = '5da07161-3770-4a4b-aa43-418cbbb627cf'
resource_group = 'aml-dev-rg'
workspace = 'aml-dev'


# os.environ.setdefault('subscription_id', subscription_id)
# os.environ.setdefault('resource_group', resource_group)
# os.environ.setdefault('workspace', workspace)

# subscription_id = os.environ.get('subscription_id')
# resource_group = os.environ.get('resource_group')
# workspace = os.environ.get('workspace')

print(subscription_id)

5da07161-3770-4a4b-aa43-418cbbb627cf


In [30]:
#import required libraries
import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml.entities import Environment, BuildContext

In [31]:
import os

# Create a folder for the experiment files
script_folder = './src/'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'src folder created')


# Create a folder for the experiment files
script_folder = './src/prep'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'prep folder created')

script_folder = './src/train'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'train folder created')

script_folder = './src/eval'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'eval folder created')

script_folder = './src/deploy'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'deploy folder created')

script_folder = './src/pipeline'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'pipeline folder created')

script_folder = './src/conda-yamls'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'conda-yamls folder created')


./src/ src folder created
./src/prep prep folder created
./src/train train folder created
./src/eval eval folder created
./src/deploy deploy folder created
./src/pipeline pipeline folder created
./src/conda-yamls conda-yamls folder created


In [32]:
%%writefile ./src/conda-yamls/pipeline_conda_env.yml
name: job_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.8.5
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow
  - azureml-mlflow
  - azure-ai-ml
  - mltable

Overwriting ./src/conda-yamls/pipeline_conda_env.yml


# Data Preperation

## Connecting to your workspace

In [33]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    # This will open a browser page for
    credential = InteractiveBrowserCredential()

In [41]:
import json

def force_login():
    
    config_path = "../.azureml/config.json"
    os.makedirs(os.path.dirname(config_path), exist_ok=True)
        
    client_config = {
        "subscription_id": subscription_id,
        "resource_group": resource_group,
        "workspace_name": workspace,
    }
    
    with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
    ml_client = MLClient.from_config(credential=credential, path=config_path)
    
    print(client_config)
    
force_login()

Found the config file in: ../.azureml/config.json


{'subscription_id': '5da07161-3770-4a4b-aa43-418cbbb627cf', 'resource_group': 'aml-dev-rg', 'workspace_name': 'aml-dev'}


In [44]:
#connect to the workspace
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": subscription_id,
        "resource_group": resource_group,
        "workspace_name": workspace,
    }

    if client_config["subscription_id"].startswith("<"):
        print(
            "please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <AML_WORKSPACE_NAME> in notebook cell"
        )
        raise ex
    else:  # write and reload from config file
        import json, os

        config_path = "../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/mmcompute-instance/code/Users/memasanz/mlops/.azureml/config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f6ed6f87c70>,
         subscription_id=5da07161-3770-4a4b-aa43-418cbbb627cf,
         resource_group_name=aml-dev-rg,
         workspace_name=aml-dev)


In [45]:
# import mlflow
# client = mlflow.tracking.MlflowClient()
# client.list_artifacts("000d6ec3-533a-4ed7-8644-667da5d7a8d8")

In [46]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

In [47]:
df= pd.read_csv('./data/titanic.csv')
print(df.shape)
print(df.columns)

try:
    registered_data_asset = ml_client.data.get(name='titanic_raw', version=1)
    print('data asset is registered')
except:
    print('register data asset')
    my_data = Data(
        path="./data/titanic.csv",
        type=AssetTypes.URI_FILE,
        description="Titanic CSV",
        name="titanic_raw",
        version="1",
    )

    ml_client.data.create_or_update(my_data)

(891, 12)
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
data asset is registered


## Preparing Scripts for Training Pipelines

## Prep Data.py

In [48]:
%%writefile ./src/prep/prep.py

import argparse
import pandas as pd

parser = argparse.ArgumentParser("prep")
parser.add_argument("--raw_data", type=str, help="Path to raw data")
parser.add_argument("--prep_data", type=str, help="Path of prepped data")
args = parser.parse_args()

print(args.raw_data)
print(args.prep_data)

df = pd.read_csv(args.raw_data + '/titanic.csv')

df['Age'] = df.groupby(['Pclass', 'Sex'])['Age'].apply(lambda x: x.fillna(x.median()))
df['Sex']= df['Sex'].apply(lambda x: x[0] if pd.notnull(x) else 'X')
df['Loc']= df['Cabin'].apply(lambda x: x[0] if pd.notnull(x) else 'X')
df.drop(['Cabin', 'Ticket'], axis=1, inplace=True)
df['Embarked'] = df['Embarked'].fillna('S')
df.loc[:,'GroupSize'] = 1 + df['SibSp'] + df['Parch']

LABEL = 'Survived'
df_train = df
df = df_train.drop(['Name','SibSp', 'Parch', 'PassengerId'], axis=1)

df.to_csv(args.prep_data)

Overwriting ./src/prep/prep.py


## Create train.py file

In [49]:
%%writefile ./src/train/train.py
import os
import mlflow
import argparse
from mlflow.tracking import MlflowClient
import pandas as pd
import matplotlib.pyplot as plt
import mlflow
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score,roc_curve
import joblib
import mlflow
import mlflow.sklearn
import shutil
from sklearn.metrics import accuracy_score, precision_score, recall_score


# define functions
def main(args):
    # enable auto logging
    current_run = mlflow.start_run()
    mlflow.sklearn.autolog()

    # read in data
    print('about to read file:' + args.prep_data)
    df = pd.read_csv(args.prep_data)
    #model, X_test = model_train('Survived', df, args.randomstate)
    model, X_test, y_test = model_train('Survived', df, 0)
    
    model_file = os.path.join(args.model_output, 'titanic_model.pkl')
    joblib.dump(value=model, filename=model_file)
    
    os.makedirs("outputs", exist_ok=True)
    y_test.to_csv('outputs/Y_test.csv', index = False)
    X_test.to_csv( 'outputs/X_test.csv', index = False)
    shutil.copytree('./outputs/', args.test_data, dirs_exist_ok=True)
    #maybe do mlflow logmodel
    mlflow.sklearn.log_model(model, "championmodel")

    

    

def model_train(LABEL, df, randomstate):
    print('df.columns = ')
    print(df.columns)
    y_raw           = df[LABEL]
    columns_to_keep = ['Embarked', 'Loc', 'Sex','Pclass', 'Age', 'Fare', 'GroupSize']
    X_raw           = df[columns_to_keep]
    
    X_raw['Embarked'] = X_raw['Embarked'].astype(object)
    X_raw['Loc'] = X_raw['Loc'].astype(object)
    X_raw['Loc'] = X_raw['Sex'].astype(object)
    X_raw['Pclass'] = X_raw['Pclass'].astype(float)
    X_raw['Age'] = X_raw['Age'].astype(float)
    X_raw['Fare'] = X_raw['Fare'].astype(float)
    X_raw['GroupSize'] = X_raw['GroupSize'].astype(float)
    


    print(X_raw.columns)
     # Train test split
    X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, test_size=0.2, random_state=args.randomstate)
    
    #use Logistic Regression estimator from scikit learn
    lg = LogisticRegression(penalty='l2', C=1.0, solver='liblinear')
    preprocessor = buildpreprocessorpipeline(X_train)
    
    #estimator instance
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', lg)], verbose=True)

    model = clf.fit(X_train, y_train)
    
    print('type of X_test = ' + str(type(X_test)))
          
    y_pred = model.predict(X_test)
    
    print('*****X_test************')
    print(X_test)
    
    metrics = mlflow.sklearn.eval_and_log_metrics(model, X_test, y_test, prefix="test_")
    
    #get the active run.
    run = mlflow.active_run()
    print("Active run_id: {}".format(run.info.run_id))
    MlflowClient().log_metric(run.info.run_id, "metric", 0.22)

    
    return model, X_test, y_test

    #mlflow.end_run()


def buildpreprocessorpipeline(X_raw):

    categorical_features = X_raw.select_dtypes(include=['object', 'bool']).columns
    numeric_features = X_raw.select_dtypes(include=['float','int64']).columns

    categorical_transformer = Pipeline(steps=[('onehotencoder', 
                                               OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore'))])


    numeric_transformer1 = Pipeline(steps=[('scaler1', SimpleImputer(missing_values=np.nan, strategy = 'mean'))])
    

    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric1', numeric_transformer1, numeric_features),
            ('categorical', categorical_transformer, categorical_features)], remainder='drop')
    
    return preprocessor



def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--prep_data", default="data", type=str, help="Path to prepped data, default to local folder")
    parser.add_argument("--input_file_name", type=str, default="titanic.csv")
    parser.add_argument("---randomstate", type=int, default=42)
#     
    parser.add_argument("--model_output", type=str, help="Path of output model")
    parser.add_argument("--test_data", type=str,)

    # parse args
    args = parser.parse_args()
    print(args.prep_data)
    print(args.input_file_name)
    print(args.randomstate)
    print(args.model_output)
    print(args.test_data)
    # return args
    return args


# run script
if __name__ == "__main__":
    # parse args
    args = parse_args()

    # run main function
    main(args)

Overwriting ./src/train/train.py


In [50]:
## Evaluate Model

In [64]:
%%writefile ./src/eval/evaluatemodel.py
import os
import mlflow
import argparse
from mlflow.tracking import MlflowClient
import pandas as pd
import matplotlib.pyplot as plt
import mlflow
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score,roc_curve
import joblib
import mlflow
import mlflow.sklearn
import shutil
from sklearn.metrics import accuracy_score, precision_score, recall_score
from azureml.core import Run
from azureml.core import Model

    
    
# define functions
def main(args):
    
    model_name = args.model_name
    
    run = Run.get_context()
    ws = run.experiment.workspace
    run_id = run.id
    print('run_id =' + run_id)
    model_list = Model.list(ws, name=model_name, latest=True)
    first_registration = len(model_list)==0
    current_model = None
        
    # read in data
    print('about to read file:' + args.test_data)
    X_test = pd.read_csv(args.test_data + '/X_test.csv')
    df_y_test = pd.read_csv(args.test_data + '/Y_test.csv')
    y_test  = df_y_test.values.flatten()
    #load champion model
    model_file = os.path.join(args.model_folder, 'titanic_model.pkl')
    champion_model = joblib.load(model_file)
    
    y_pred_current = champion_model.predict(X_test)
    print('y_pred_current')
    print(y_pred_current)
    print('y_test')
    print(y_test)
    
    champion_auc = roc_auc_score(y_test,y_pred_current)
    print('champion_auc:' , champion_auc)
    
    champion_acc = np.average(y_pred_current == y_test)
    print('champion_acc:', champion_acc)


    
    try:
        current_model_aml = Model(ws,args.model_name)
        os.makedirs("current_model", exist_ok=True)
        current_model_aml.download("current_model",exist_ok=True)
        current_model = mlflow.sklearn.load_model(os.path.join("current_model",args.model_name))
    except:
        print('no model register with name' + args.model_name)
        pass
    
    if current_model:
        y_pred_current = current_model.predict(X_test)
        current_acc = np.average(y_pred_current == y_test)
        print('current_acc:', current_acc)
        if champion_acc >= current_acc:
            print('better model found, registering')
            mlflow.sklearn.log_model(champion_model,args.model_name)
            model_uri = f'runs:/{run_id}/{args.model_name}'
            mlflow.register_model(model_uri,args.model_name)
            
        else:
            print('current model performs better than champion model ')
            print('champion_acc:', champion_acc)
            print('current_acc:', current_acc)
    else:
        print('no current model')
        print("First time model train, registering")
        mlflow.sklearn.log_model(champion_model,args.model_name)
        model_uri = f'runs:/{run_id}/{args.model_name}'
        mlflow.register_model(model_uri,args.model_name)
        
        print('hello')

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--test_data", default="data", type=str, help="Path to test data")
    parser.add_argument("--model_folder", default="data", type=str, help="Path to model data")
    parser.add_argument("--model_name",default='mmchapter9titanic',type=str, help="Name of the model in workspace")

    parser.add_argument("--model_deployment_files", default="data", type=str, help="Path to model data")
    # parse args
    args = parser.parse_args()
    
    print(args.test_data)
    print(args.model_folder)
    print(args.model_name)
        
    return args


# run script
if __name__ == "__main__":
    # parse args
    args = parse_args()

    # run main function
    main(args)


Overwriting ./src/eval/evaluatemodel.py


In [52]:
#want to add:

# settings:
#   default_datastore: azureml:workspaceblobstore
#   continue_on_step_failure: false
#   force_rerun: true

In [53]:
%%writefile ./src/pipeline/aml_test_pipeline.yml

$schema: https://azuremlschemas.azureedge.net/latest/pipelineJob.schema.json
type: pipeline
display_name: hello_pipeline
jobs:
  hello_job:
    command: echo "hello"
    environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
    compute: azureml:cluster1
  world_job:
    command: echo "world"
    environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
    compute: azureml:cluster1
            

Overwriting ./src/pipeline/aml_test_pipeline.yml


In [76]:
%%writefile ./src/pipeline/aml_train_and_eval_pipeline.yml

$schema: https://azuremlschemas.azureedge.net/latest/pipelineJob.schema.json
type: pipeline
display_name: Training_and_eval_pipeline
compute: azureml:cpu-cluster

jobs:
  prep_job:
    type: command
    code: ../prep
    command: >-
      python prep.py 
      --raw_data ${{inputs.raw_data}}
      --prep_data ${{outputs.prep_data}}
    inputs:
      raw_data:
        type: uri_folder
        path: azureml:titanic_raw:1
        mode: ro_mount
    outputs:
      prep_data:
        type: uri_file
        path: azureml://datastores/workspaceblobstore/paths/titanic_prep_data/titanic_prepped.csv
        mode: rw_mount
    environment:
      conda_file: ../conda-yamls/pipeline_conda_env.yml
      image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest

    
  train_job:
    type: command
    inputs:
      prep_data: ${{parent.jobs.prep_job.outputs.prep_data}}
    outputs:
      model_output:
        type: uri_folder
        path: azureml://datastores/workspaceblobstore/paths/titanic_model_data/
        mode: rw_mount
      test_data: 
        type: uri_folder
        path: azureml://datastores/workspaceblobstore/paths/titanic_test_data/
        mode: rw_mount
    code: ../train
    environment:
      conda_file: ../conda-yamls/pipeline_conda_env.yml
      image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest
    compute: azureml:cpu-cluster
    command: >-
      python train.py 
      --prep_data ${{inputs.prep_data}} 
      --model_output ${{outputs.model_output}}
      --test_data ${{outputs.test_data}}

  eval_job:
    type: command
    inputs:
      test_data: ${{parent.jobs.train_job.outputs.test_data}}
      model_folder: ${{parent.jobs.train_job.outputs.model_output}}
    outputs:
      model_deployment_files:
        type: uri_folder
        path: azureml://datastores/workspaceblobstore/paths/titanic_model_deployment_files/
        mode: rw_mount
    code: ../eval
    environment:
      conda_file: ../conda-yamls/pipeline_conda_env.yml
      image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest
    compute: azureml:cpu-cluster
    command: >-
      python evaluatemodel.py 
      --test_data ${{inputs.test_data}} 
      --model_folder ${{inputs.model_folder}}
      --model_deployment_files $${{outputs.model_deployment_files}}

Overwriting ./src/pipeline/aml_train_and_eval_pipeline.yml


# Create Deployment files

### Be sure to change your endpoint name to be something that wil be unique

In [33]:
%%writefile ./src/deploy/create-endpoint.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json 
name: chapter9titanicendpoint3
auth_mode: key 

Overwriting ./src/deploy/create-endpoint.yml


### Be sure to change your endpoint name to match the endpoint name in the `create-endpoint.yml` file

In [34]:
%%writefile ./src/deploy/model_deployment.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json 
name: green
endpoint_name: chapter9titanicendpoint3
model: azureml:mmchapter9titanic@latest 
instance_type: Standard_DS2_v2 
instance_count: 2  
    

Overwriting ./src/deploy/model_deployment.yml


In [35]:
%%writefile ./src/AzureDevOpsPipeline.yml

resources:
  containers:
  - container: mlops
    image: mcr.microsoft.com/mlops/python:latest

pr: none
trigger:
  branches:
    include:
    - main

variables:
- group: devops-variable-group-dev
- group: devops-variable-group-qa
- name: model_name
  value: mmchapter9titanic
- name: ENDPT_NAME
  value: chapter9titanicendpoint3
- name: BLUE_DEPLOYMENT_NAME
  value: blue
- name: GREEN_DEPLOYMENT_NAME
  value: green



pool:
  vmImage: ubuntu-latest

stages:        
- stage: 'testonly'
  variables:
  - group: devops-variable-group-qa
  displayName: 'QAPromoteModel'
  jobs:
  - job: "RegisterModelQA"
    steps:
      - task: AzureCLI@1
        env:
          wsName: $(wsName)
          resourceGroup: $(resourceGroup)
          location: $(location)
        inputs:
          azureSubscription: aml-qa
          scriptLocation: inlineScript
          workingDirectory: '$(Build.SourcesDirectory)'
          inlineScript: |
            export NEW_DEPLOYMENT_NAME=deployment`echo $(date +"%s")`
            echo $NEW_DEPLOYMENT_NAME
            
            az extension add -n ml -y
            az version
            #check if the endpoint already exists or not.
            #if it doesn't exist, create 2 online deployments, so you can always swap between 2, one if for prod, and one is for the swap

            ENDPOINT_EXISTS=$(az ml online-endpoint list -g $(resourceGroup) -w $(wsName) -o tsv --query "[?name=='$ENDPT_NAME'][name]" |  wc -l)
            if [[ ENDPOINT_EXISTS -ne 1 ]]; then
                echo "endpoint does not exists"
                az ml online-endpoint create --file Chapter09/src/deploy/create-endpoint.yml -g $(resourceGroup) -w $(wsName)
                #deploy to both blue and green, so there will always be a blue and a green to swap between
                #az ml online-deployment create --name $BLUE_DEPLOYMENT_NAME -f Chapter09/src/deploy/model_deployment.yml -g $(resourceGroup) -w $(wsName)
                #arbitrarily deploy the green endpoint since neither exists
                echo "creating online deployment"
                az ml online-deployment create --name $NEW_DEPLOYMENT_NAME -f Chapter09/src/deploy/model_deployment.yml -g $(resourceGroup) -w $(wsName) 
                echo "updating online endpoint tags"
                az ml online-endpoint update -n $ENDPT_NAME --set tags.prod=$NEW_DEPLOYMENT_NAME  --traffic "$NEW_DEPLOYMENT_NAME=100" -g $(resourceGroup) -w $(wsName)
                exit 0
            else
                echo "endpoint exists, get the prod one "
                PROD_DEPLOYMENT=$(az ml online-endpoint show -n $ENDPT_NAME -g $(resourceGroup) -w $(wsName) -o tsv --query "tags.prod")
                echo $PROD_DEPLOYMENT
                az ml online-deployment create -g $(resourceGroup) -w $(wsName) --name $NEW_DEPLOYMENT_NAME -f Chapter09/src/deploy/model_deployment.yml 
                az ml online-endpoint update -g $(resourceGroup) -w $(wsName) -n $ENDPT_NAME --set tags.prod=$NEW_DEPLOYMENT_NAME  --traffic "$NEW_DEPLOYMENT_NAME=100"
                az ml online-deployment delete -g $(resourceGroup) -w $(wsName) --endpoint $ENDPT_NAME --name $PROD_DEPLOYMENT --yes --no-wait
            fi

Overwriting ./src/AzureDevOpsPipeline.yml


In [110]:
#az ml job create --file Chapter09/src/pipeline/aml_train_and_eval_pipeline.yml --stream --set settings.force_rerun=True

In [1]:
# %%writefile ./src/AzureDevOpsPipeline.yml

# resources:
#   containers:
#   - container: mlops
#     image: mcr.microsoft.com/mlops/python:latest

# pr: none
# trigger:
#   branches:
#     include:
#     - main

# variables:
# - group: devops-variable-group-dev
# - group: devops-variable-group-qa
# - name: model_name
#   value: mmchapter9titanic
# - name: online_endpoint_name
#   value: chapter9titanicendpoint


# pool:
#   vmImage: ubuntu-latest

# stages:
# - stage: 'DevRunPipline'
#   variables:
#   - group: devops-variable-group-dev
#   displayName: 'DevTrainingPipeline'
#   jobs:
#   - job: "TrainingPipeline"
#     steps:  
#       - task: AzureCLI@1
#         env:
#           wsName: $(wsName)
#           resourceGroup: $(resourceGroup)
#           location: $(location)
#         inputs:
#           azureSubscription: aml-dev
#           scriptLocation: inlineScript
#           workingDirectory: '$(Build.SourcesDirectory)'
#           inlineScript: |
#             echo "files:"
#             ls
#             az version
#             az extension add -n ml -y
#             az version
#             az configure --defaults group=$(resourceGroup) workspace=$(wsName) location=$(location)
#             az ml model list -w $(wsName) -g $(resourceGroup)  -n $(model_name) --query "[0].version" -o tsv
#             echo "##vso[task.setvariable variable=modelversion;isOutput=true]$(az ml model list -w $(wsName) -g $(resourceGroup)  -n $(model_name) --query '[0].version' -o tsv)"
#         name: 'setversion'
#         displayName: 'Get Initial Model Version'
            
#       - task: Bash@3
#         inputs:
#           workingDirectory: '$(Build.SourcesDirectory)'
#           targetType: 'inline'
#           script: |
#             echo 'modelversion'
#             echo $(setversion.modelversion)     
            
#       - task: AzureCLI@1
#         timeoutInMinutes: 30
#         env:
#           wsName: $(wsName)
#           resourceGroup: $(resourceGroup)
#           location: $(location)
#         inputs:
#           azureSubscription: aml-dev
#           scriptLocation: inlineScript
#           workingDirectory: '$(Build.SourcesDirectory)'
#           inlineScript: |
#             echo "initial model version"
#             echo $(setversion.modelversion)
#             echo "files"
#             ls
#             #az ml job create --file Chapter09/src/pipeline/aml_train_and_eval_pipeline.yml --stream 
#             az ml job create --file Chapter09/src/pipeline/aml_train_and_eval_pipeline.yml --stream --set settings.force_rerun=True
#         displayName: 'Training Pipeline'
            
#       - task: AzureCLI@1
#         env:
#           wsName: $(wsName)
#           subscriptionId: $(subscriptionId)
#           resourceGroup: $(resourceGroup)
#           location: $(location)
#         inputs:
#           azureSubscription: aml-dev
#           scriptLocation: inlineScript
#           workingDirectory: '$(Build.SourcesDirectory)'
#           inlineScript: |
#             echo "files:"
#             ls
#             az version
#             az configure --defaults group=$(resourceGroup) workspace=$(wsName) location=$(location)
#             az ml model list -w $(wsName) -g $(resourceGroup)  -n $(model_name) --query "[0].version" -o tsv
#             echo "##vso[task.setvariable variable=finalmodelversion;isOutput=true]$(az ml model list -w $(wsName) -g $(resourceGroup)  -n $(model_name) --query '[0].version' -o tsv)"
#             echo "##vso[task.setvariable variable=devResourceGroup;isOutput=true]$(resourceGroup)"
#             echo "##vso[task.setvariable variable=devWsName;isOutput=true]$(wsName)"
#             echo "##vso[task.setvariable variable=devLocation;isOutput=true]$(location)"
#         name: 'setfinalversion'
#         displayName: 'Get Final Model Version'
            
#       - task: Bash@3
#         inputs:
#           azureSubscription:
#           workingDirectory: '$(Build.SourcesDirectory)'
#           targetType: 'inline'
#           script: |
#             echo 'initial model version'$(setversion.modelversion)
#             echo 'final model version' $(setfinalversion.finalmodelversion)
            
# - stage: 'QAPromoteModel'
#   dependsOn: DevRunPipline
#   variables:
#   - group: devops-variable-group-qa
#   displayName: 'QAPromoteModel'
#   jobs:
#   - job: "RegisterModelQA"
#     variables:
#       vardevResourceGroup: $[ stageDependencies.DevRunPipline.TrainingPipeline.outputs['setfinalversion.devResourceGroup'] ]
#       vardevWsName: $[ stageDependencies.DevRunPipline.TrainingPipeline.outputs['setfinalversion.devWsName'] ]
#       vardevLocation: $[ stageDependencies.DevRunPipline.TrainingPipeline.outputs['setfinalversion.devLocation'] ]
#       vardevModelVersion: $[ stageDependencies.DevRunPipline.TrainingPipeline.outputs['setfinalversion.finalmodelversion'] ]
#     steps:            
#       - task: AzureCLI@1
#         env:
#           wsName: $(wsName)
#           resourceGroup: $(resourceGroup)
#           location: $(location)
#         inputs:
#           azureSubscription: aml-dev
#           scriptLocation: inlineScript
#           workingDirectory: '$(Build.SourcesDirectory)'
#           inlineScript: |
#             az extension add -n ml -y
#             az version
#             echo "model version"
#             echo $(vardevModelVersion)
#             az ml model list -w $(vardevWsName) -g $(vardevResourceGroup)  -n $(model_name) --query "[0].version" -o tsv
#             az ml model download  -w $(vardevWsName) -g $(vardevResourceGroup)  -n $(model_name) -v $(vardevModelVersion)
#             ls
#         name: 'downloadmodel'
#         displayName: 'downloadmodel'
            
#       - task: AzureCLI@1
#         env:
#           wsName: $(wsName)
#           resourceGroup: $(resourceGroup)
#           location: $(location)
#         inputs:
#           azureSubscription: aml-qa
#           scriptLocation: inlineScript
#           workingDirectory: '$(Build.SourcesDirectory)'
#           inlineScript: |
#             echo "files:"
#             ls
#             echo "model version" $(vardevModelVersion)
#             az configure --defaults group=$(resourceGroup) workspace=$(wsName) location=$(location)
#             az ml model create --name $(model_name) -v $(vardevModelVersion) --path ./$(model_name)/$(model_name) --type mlflow_model -g $(resourceGroup) -w $(wsName)
#         name: 'registermodel'
#         displayName: 'registermodel'
            
#  

Overwriting ./src/AzureDevOpsPipeline.yml


In [ ]:
#       - task: AzureCLI@1
#         inputs:
#           azureSubscription: aml-qa
#           scriptLocation: inlineScript
#           workingDirectory: '$(Build.SourcesDirectory)'
#           inlineScript: |
#             echo $(checkendpointexists.result)
#             echo $(checkendpointexists.onlinedeploymentresult)
#             if [[ $(az ml online-endpoint show --name $(online_endpoint_name)  -g $(resourceGroup) -w $(wsName) --query provisioning_state) == 'Succeeded' ]]
#             then
#               echo 'already exists'
#             else
#               az ml online-endpoint create --file Chapter09/src/deploy/create-endpoint.yml -g $(resourceGroup) -w $(wsName)
#             fi
#         name: 'deployonlineendpoint'
#         displayName: 'Online endpoint'
            

# - task: AzureCLI@1
#         inputs:
#           azureSubscription: aml-qa
#           scriptLocation: inlineScript
#           workingDirectory: '$(Build.SourcesDirectory)'
#           inlineScript: |
#             echo $(checkendpointexists.exists)
#             echo $(checkendpointexists.result)
#             if [[ $(onlinedeploymentresult) == 0 ]]
#             then
#             {
#                 echo "update model"
#                 az ml online-deployment update --file Chapter09/src/deploy/model_deployment.yml -g $(resourceGroup) -w $(wsName)
#             }
              
#             else
#             {
#                 echo "create model"
#                 az ml online-deployment create --file Chapter09/src/deploy/model_deployment.yml -g $(resourceGroup) -w $(wsName)
#             }
              
#             fi
#         name:'deploymodel'
#         displayName: 'DeployModel'
            
            
            
            
#       - task: AzureCLI@1
#         env:
#           wsName: $(wsName)
#           resourceGroup: $(resourceGroup)
#           location: $(location)
#         inputs:
#           azureSubscription: aml-qa
#           scriptLocation: inlineScript
#           workingDirectory: '$(Build.SourcesDirectory)'
#           inlineScript: |
#             echo "deploy end point:"
#             az ml online-deployment create --file Chapter09/src/deploy/model_deployment.yml -g $(resourceGroup) -w $(wsName)
#         name: 'deploymodel'
#         displayName: 'DeployModel'
            
            
            
# #       - task: PowerShell@2
# #         env:
# #           wsName: $(wsName)
# #           resourceGroup: $(resourceGroup)
# #           location: $(location)
# #         inputs:
# #           azureSubscription: aml-qa
# #           scriptLocation: inlineScript
# #           workingDirectory: '$(Build.SourcesDirectory)'
# #           targetType: 'inline' #'filePath' # 'filePath' | 'inline'. Type. Default: filePath.
# #           script: |# string. Required when targetType = inline. Script. 
# #             ls
# #             echo 'test'
# #             echo "##vso[task.setvariable variable=result;isOutput=true]$(az ml online-endpoint show --name $(online_endpoint_name)  -g $(resourceGroup) -w $(wsName) --query provisioning_state | ConvertFrom-Json)"
# #             echo "##vso[task.setvariable variable=onlinedeploymentresult;isOutput=true]$(az ml online-deployment show -n green --endpoint-name $(online_endpoint_name) -g $(resourceGroup) -w $(wsName) | ConvertFrom-Json)"
# #             az ml online-endpoint show --name $(online_endpoint_name)  -g $(resourceGroup) -w $(wsName) --query provisioning_state | ConvertFrom-Json
# #             echo "deployment check"
# #             az ml online-deployment show -n green --endpoint-name $(online_endpoint_name) -g $(resourceGroup) -w $(wsName) | ConvertFrom-Json
# #         name: 'checkendpointexists'
# #         displayName: 'Get if end point exists'            
            
    
# #       - task: AzureCLI@1
# #         inputs:
# #           azureSubscription: aml-qa
# #           scriptLocation: inlineScript
# #           workingDirectory: '$(Build.SourcesDirectory)'
# #           inlineScript: |
# #             echo $(checkendpointexists.result)
# #             echo $(checkendpointexists.onlinedeploymentresult)
# #         name: 'deployonlineendpoint'
# #         displayName: 'Online endpoint'

In [ ]:
## Deploy Managed Online Endpoint


In [73]:
# import datetime

# online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# # create an online endpoint
# endpoint = ManagedOnlineEndpoint(
#     name=online_endpoint_name,
#     description="titanic online endpoint for mlflow model",
#     auth_mode="key",
#     tags={"oneline endpoint": "titanic"},
# )

In [ ]:
# %%writefile ./src/AzureDevOpsPipeline.yml

# resources:
#   containers:
#   - container: mlops
#     image: mcr.microsoft.com/mlops/python:latest

# pr: none
# trigger:
#   branches:
#     include:
#     - main

# variables:
# - group: devops-variable-group-dev
# - group: devops-variable-group-qa


# pool:
#   vmImage: ubuntu-latest

# stages:
# - stage: 'RunPipline'
#   variables:
#   - group: devops-variable-group-dev
#   displayName: 'TrainingPipeline'
#   jobs:
#   - job: "TrainingPipeline"
#     steps:
#       - task: UsePythonVersion@0
#         inputs:
#           versionSpec: '3.8'
#           addToPath: true
#       - script: |
#           python -m pip install --upgrade pip
#           pip install jupyter
#           pip install nbconvert
#           pip install --upgrade azureml-core
#           pip install --upgrade azureml-sdk[automl]
      

#       - task: AzureCLI@1
#         env:
#           wsName: $(wsName)
#           subscriptionId: $(subscriptionId)
#           resourceGroup: $(resourceGroup)
#           location: $(location)
#         inputs:
#           azureSubscription: aml-dev
#           scriptLocation: inlineScript
#           workingDirectory: '$(Build.SourcesDirectory)'
#           inlineScript: |
#             echo "files:"
#             ls
#             az version
#             az extension add -n ml -y
#             az version
#             az configure --defaults group=$(resourceGroup) workspace=$(wsName) location=$(location)
#             az ml model list -w aml-dev -g aml-dev-rg  -n mmchapter8titanic --query "[0].version" -o tsv
#             echo "##vso[task.setvariable variable=modelversion;isOutput=true]$(az ml model list -w aml-dev -g aml-dev-rg  -n mmchapter8titanic --query '[0].version' -o tsv)"
#         name: 'setversion'
#         displayName: 'Get Initial Model Version'
            
#       - task: Bash@3
#         inputs:
#           workingDirectory: '$(Build.SourcesDirectory)'
#           targetType: 'inline'
#           script: |
#             echo 'modelversion'
#             echo $(setversion.modelversion)     
            
#       - task: AzureCLI@1
#         env:
#           wsName: $(wsName)
#           subscriptionId: $(subscriptionId)
#           resourceGroup: $(resourceGroup)
#           location: $(location)
#         inputs:
#           azureSubscription: aml-dev
#           scriptLocation: inlineScript
#           workingDirectory: '$(Build.SourcesDirectory)'
#           inlineScript: |
#             echo $initalmodelversion
#             echo ls
#             az ml job create --file Chapter8/src/pipeline/aml_train_and_eval_pipeline.yml --stream --set settings.force_rerun=True
#         displayName: 'Training Pipeline'
            
#       - task: AzureCLI@1
#         env:
#           wsName: $(wsName)
#           subscriptionId: $(subscriptionId)
#           resourceGroup: $(resourceGroup)
#           location: $(location)
#         inputs:
#           azureSubscription: aml-dev
#           scriptLocation: inlineScript
#           workingDirectory: '$(Build.SourcesDirectory)'
#           inlineScript: |
#             echo "files:"
#             ls
#             az version
#             az extension add -n ml -y
#             az version
#             az configure --defaults group=$(resourceGroup) workspace=$(wsName) location=$(location)
#             az ml model list -w aml-dev -g aml-dev-rg  -n mmchapter8titanic --query "[0].version" -o tsv
#             echo "##vso[task.setvariable variable=finalmodelversion;isOutput=true]$(az ml model list -w aml-dev -g aml-dev-rg  -n mmchapter8titanic --query '[0].version' -o tsv)"
#         name: 'setfinalversion'
#         displayName: 'Get Final Model Version'
            
#       - task: Bash@3
#         inputs:
#           workingDirectory: '$(Build.SourcesDirectory)'
#           targetType: 'inline'
#           script: |
#             echo 'modelversion'
#             echo $(setversion.modelversion)
#             echo $(setfinalversion.finalmodelversion)
            
# - stage: 'DeployModel'
#   dependsOn: RunPipline
#   condition: ne(dependencies.RunPipline.outputs['TrainingPipeline.setversion.modelversion'], dependencies.RunPipline.outputs['TrainingPipeline.setversion.modelversion'])
#   jobs:
#   - job: "DeployModel"
#     steps:
#       - task: AzureCLI@1
#         env:
#           tenantId: $(tenantId)
#           servicePrincipalId: $(servicePrincipalId)
#           servicePrincipalPassword: $(servicePrincipalPassword)
#           wsName: $(wsName)
#           subscriptionId: $(subscriptionId)
#           resourceGroup: $(resourceGroup)
#           location: $(location)
#         inputs:
#           azureSubscription: dev-aml-workspace-connection
#           scriptLocation: inlineScript
#           workingDirectory: '$(Build.SourcesDirectory)'
#           inlineScript: |
#             echo "made it:"
#             echo dependencies.RunPipline.outputs['TrainingPipeline.setversion.modelversion']
#             ls
#             az version
#             az extension add -n ml -y
#             az version
#             az configure --defaults group=$(resourceGroup) workspace=$(wsName) location=$(location)
#             az ml model list -w aml-workspace -g aml-workspace-rg  -n mmchapter8titanic --query "[0].version" -o tsv

            
    

In [ ]:
## Create eval.py file

In [ ]:
# # Creating a unique endpoint name with current datetime to avoid conflicts
# import datetime

# online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# # create an online endpoint
# endpoint = ManagedOnlineEndpoint(
#     name=online_endpoint_name,
#     description="titanic online endpoint for mlflow model",
#     auth_mode="key",
#     tags={"oneline endpoint": "titanic"},
# )

In [ ]:
# %%writefile $script_folder/endpoint.yml
# $schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json
# name: titanic-managed-online-endpoint
# description: "CLI V2 titanic online endpoint for mlflow model"
# auth_mode: key
# tags : {"CLIV2": "titanic"}

In [ ]:
# %%writefile $script_folder/deployment.yml

# $schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
# name: blue
# endpoint_name: titanic-managed-online-endpoint
# model: azureml:chapter6_titanic_model:1
# code_configuration:
#   code: 
#     local_path: .
#   scoring_script: score.py
# environment: azureml:job_base_env:1
# instance_type: Standard_F2s_v2
# instance_count: 1

In [ ]:
# %%writefile $script_folder/score.py

# import os 
# import json
# import joblib
# from pandas import json_normalize
# import pandas as pd

# # Called when the service is loaded
# def init():
#     global model
#     # Get the path to the deployed model file and load it
#     model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'titanic_model.pkl')
#     model = joblib.load(model_path)

# # Called when a request is received
# def run(raw_data):
#     dict= json.loads(raw_data)
#     df = json_normalize(dict['raw_data']) 
#     y_pred = model.predict(df)
#     print(type(y_pred))
    
#     result = {"result": y_pred.tolist()}
#     return result